In [1]:
# Read trajectory data

import numpy as np
from msmbuilder.example_datasets import FsPeptide
from msmbuilder.dataset import dataset
import mdtraj as md

fs_peptide = FsPeptide()
fs_peptide.cache()
xyz = dataset(fs_peptide.data_dir + "/*.xtc",
              topology=fs_peptide.data_dir + '/fs-peptide.pdb',
              stride=10)

temp = xyz[0]
_, num_atoms, num_axis = temp.xyz.shape
reference_frame = temp.slice(0, copy=True)
num_features = num_atoms*num_axis;
pre_X = [np.reshape(traj.xyz, (traj.superpose(reference_frame).xyz.shape[0],num_features)) for traj in xyz]
X = np.concatenate(pre_X)
num_frames = X.shape[0]

C:\Anaconda2\lib\site-packages\mdtraj\formats\pdb\pdbfile.py:196: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn('Unlikely unit cell vectors detected in PDB file likely '


In [ ]:
# Load isomap data

# Load isomap cluster labels
# Ensure that the smallest cluster label is 0!
iso_label = np.load('./sherlock_data/isomap_clustering_labels_30_40_97.dat')

# Load isomap coordinate
iso_coord = np.load('./sherlock_data/isomap_coordinates_30_40_97.dat')

# Load isomap cluster centroids
iso_cent = np.load('./sherlock_data/isomap_clusters_RD_30_40_97.dat')

# Load raw X,Y,Z

In [ ]:
# create cluster adjacency matrix

cluster_num = len(np.unique(iso_label))
n = 4
cluster_adj = np.zeros((cluster_num, cluster_num))

# create full adjacency matrix
for i in range(0,cluster_num-1):
    for j in range(i+1, cluster_num):
        cluster_adj[i,j] = np.linalg.norm(iso_cent[i]-iso_cent[j])

cluster_adj = cluster_adj + np.transpose(cluster_adj)
# prunce adjacency matrix
for i in range(0, cluster_num):
    ind = np.argpartition(cluster_adj[i], -cluster_num+n+1)[-cluster_num+n+1:]
    for j in ind:
        cluster_adj[i,j] = 0



In [ ]:
#visualize how the clusters are connected

def show_graph(adjacency_matrix):
    # given an adjacency matrix use networkx and matlpotlib to plot the graph
    import networkx as nx
    import matplotlib.pyplot as plt

    rows, cols = np.where(adjacency_matrix > 0)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    # nx.draw(gr) # edited to include labels
    nx.draw_networkx(gr)
    # now if you decide you don't want labels because your graph
    # is too busy just do: nx.draw_networkx(G,with_labels=False)
    plt.show()

show_graph(cluster_adj)

In [ ]:
# create a dictionary that describes the connections of all the frames
n = 4
edges = {}
for i in range(0,cluster_num):
    print i
    cluster_frame = np.where(iso_label == i)[0] #get index of all frames in cluster i
    
    neighbor_cluster = np.where(cluster_adj[i])[0] #find connected clusters to cluster i
    neighbor_frame = np.where([x in neighbor_cluster for x in iso_label])[0] #get infex of all frames in neighboring clusters

    all_frame = np.concatenate((cluster_frame, neighbor_frame))
    length = len(all_frame)
    
    # calculate distances to all neighbors from each frame in cluster i
    for i, frame_i in enumerate(cluster_frame):
        temp = np.zeros((length))
        for j, frame_j in enumerate(all_frame): #can make this more efficient
            temp[j] = np.linalg.norm(iso_coord[frame_i]-iso_coord[frame_j]) #edjes are based on isomap distances
        temp[i] = np.nan
        # find the n nearest frames to frame_i
        nearest = np.argpartition(temp, n)[:n]
        edges[frame_i] = [(all_frame[idx], temp[idx])for idx in nearest]
   

In [ ]:
# save the dictionary
temp = {}
for a, x in edges.iteritems():
    temp[str(a)] = [(str(f),str(x))for f,x in x]


import json
with open('./dict/edges.json', 'w') as fp:
    json.dump(temp, fp)
    

In [2]:
import json
# open the dictionary
with open('./dict/edges.json', 'r') as fp:
    test = json.load(fp)

edges  = {}
for a,x in test.iteritems():
    edges[int(a)] = [(int(tup[0]), float(tup[1])) for tup in x]


In [3]:
# delete all trajectories

import sys
import os
from os import listdir
from os.path import join

dir = './trajectories/'
test=os.listdir(dir)

for item in test:
    if item.endswith('.xtc'):
        os.remove(join(dir, item))

In [5]:
# generate frame orders
def random_frame(neighbors):
    prob = []
    choice = []
    for (frame, dist) in neighbors:
        prob.append(dist)
        choice.append(frame)
    prob = np.array(prob)/sum(prob)
    return np.random.choice(choice, p = prob)

def generate_md_traj(graph_dict, X, num_traj, length = -1):
    seed = np.linspace(0, len(graph_dict)-1, num_traj)
    if length == -1:
            length = 1000
    for k in range(0,num_traj):
        traj = [int(seed[k])]
        for i in range(0, length-1):
            neighbor = graph_dict[traj[i]]
            traj.append(random_frame(neighbor))
        #print traj

        our_traj = np.reshape(X[traj,:], (len(traj), len(X[0])/3, 3))
        md_traj = md.Trajectory(our_traj, md.load(fs_peptide.data_dir + '/fs-peptide.pdb').topology)
        md_traj.save_xtc('./trajectories/' + str(k+1)+'.xtc')

import warnings
warnings.filterwarnings('ignore')
generate_md_traj(edges, X, 100, 100)